<a href="https://colab.research.google.com/github/nicolasyassuda/super-comp/blob/main/Aula11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%%writefile calcula_variancia.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/iterator/constant_iterator.h>
#include <iostream>
#include <fstream>
#include <cmath>

// Functor para calcular a diferença entre elementos consecutivos
struct diff_functor {
    __host__ __device__
    float operator()(const float& a, const float& b) const {
        return b - a;
    }
};

// Functor para calcular o quadrado da diferença entre um valor e a média
struct square_diff_functor {
    const float mean;

    square_diff_functor(float _mean) : mean(_mean) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

// Função para ler o arquivo e preencher o vetor
thrust::host_vector<float> read_stock_data(const std::string& filename) {
    std::ifstream file(filename);
    thrust::host_vector<float> stocks;
    float value;

    if (file.is_open()) {
        while (file >> value) {
            stocks.push_back(value);
        }
        file.close();
    } else {
        std::cerr << "Erro ao abrir o arquivo." << std::endl;
    }

    return stocks;
}

int main() {
    // Ler o arquivo de texto com os preços de ações
    std::string filename = "stocks-google.txt";
    thrust::host_vector<float> h_stocks = read_stock_data(filename);

    if (h_stocks.size() < 2) {
        std::cerr << "Arquivo com poucos dados para calcular diferenças diárias." << std::endl;
        return 1;
    }

    // Copiar o vetor para o device
    thrust::device_vector<float> d_stocks = h_stocks;

    // Vetor de saída com um elemento a menos
    thrust::device_vector<float> d_ganho_diario(d_stocks.size() - 1);

    // Calcular a diferença diária
    thrust::transform(d_stocks.begin(), d_stocks.end() - 1,  // Intervalo do primeiro vetor (stocks[i])
                      d_stocks.begin() + 1,                  // Intervalo do segundo vetor (stocks[i+1])
                      d_ganho_diario.begin(),                // Vetor de saída (ganho_diario[i])
                      diff_functor());                       // Functor de diferença

    // Calcular a soma total do vetor de ganho_diario
    float soma = thrust::reduce(d_ganho_diario.begin(), d_ganho_diario.end(), 0.0f, thrust::plus<float>());

    // Calcular a média
    float media = soma / d_ganho_diario.size();

    // Usar thrust::transform_reduce para calcular a variância
    float variancia = thrust::transform_reduce(
        d_ganho_diario.begin(), d_ganho_diario.end(),   // Intervalo do vetor de diferenças
        square_diff_functor(media),                     // Functor para calcular o quadrado da diferença
        0.0f,                                           // Valor inicial da redução
        thrust::plus<float>()                           // Operador de soma
    );

    // Dividir pela quantidade de elementos para obter a variância
    variancia /= d_ganho_diario.size();

    // Exibir o resultado
    std::cout << "A variância das diferenças diárias é: " << variancia << std::endl;

    return 0;
}


Overwriting calcula_variancia.cu


In [6]:
%%writefile stocks-google.txt
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00
1015.24
1020.50
1022.00
1018.75
1025.00
1028.50
1031.75
1030.00
1035.00
1032.50
1038.00
1032.75
1039.00
1040.50
1043.00
1045.25
1040.00
1038.50
1042.25
1047.75
1049.00
1051.50
1054.00
1058.25
1060.50
1065.75
1068.00
1070.50
1073.25
1075.75
1078.00
1080.25
1082.50
1086.00
1088.25
1090.75
1093.00
1096.50
1098.75
1101.00
1105.50
1108.75
1110.00
1113.25
1116.50
1118.75
1122.00
1124.50
1127.00
1130.25
1132.75
1135.50
1138.00
1140.25
1143.00
1145.50
1148.00
1150.25
1153.75
1156.00
1158.50
1160.75
1164.25
1167.50
1170.00
1173.25
1176.00
1178.75
1182.00
1185.25
1188.00
1191.50
1194.00
1196.50
1198.75
1202.00
1204.25
1208.50
1211.00
1214.75
1218.00
1220.25
1223.50
1226.00
1229.25
1232.50
1235.00
1238.75
1241.00
1243.50
1246.00
1249.25
1251.50
1255.00
1258.25
1261.00
1264.75
1268.50
1271.00
1274.25
1277.75
1280.00
1283.50
1286.75
1290.25
1293.00
1296.50
1300.00
1302.50
1305.75
1309.00
1312.50
1315.00
1318.25
1320.75
1324.00
1327.50
1330.00
1333.25
1335.50
1338.75
1342.00
1345.25
1348.00
1350.50
1354.00
1356.75
1359.50
1362.25
1365.00
1368.75
1372.00
1375.50
1378.00
1381.25
1384.75
1387.50
1390.00
1393.25
1396.50
1399.00
1402.75
1405.00
1408.50
1411.75
1415.25
1418.00
1421.50
1425.00
1428.25
1430.50
1433.75
1437.00
1440.25
1443.50
1446.00
1449.75
1453.00
1456.50
1460.00
1462.50
1465.75
1469.00
1472.25
1475.50
1478.00
1481.25
1484.75
1488.00
1491.50
1494.75
1497.50
1501.00
1504.25
1507.50
1510.75
1514.00
1516.50
1519.75
1522.00
1525.50
1529.00
1532.25
1535.50
1539.00
1542.25
1545.00
1548.75
1552.00
1555.50
1558.75
1561.50
1565.00
1568.25
1571.75
1574.00
1577.25
1580.50
1583.75
1587.00
1590.25
1593.50
1596.75
1600.00
1603.25
1606.50
1609.75
1613.00
1616.25
1619.50
1623.00
1626.25
1629.50
1632.75
1636.00
1639.50
1642.75
1645.00
1648.25
1652.00
1655.25
1658.50
1661.75
1665.00
1668.25
1671.50
1674.75
1678.00
1681.25
1684.50
1687.75
1690.00
1693.25
1696.50
1699.75
1703.00
1706.25
1709.50
1712.75
1716.00
1719.25
1722.50
1726.00
1729.25
1732.50
1735.75
1739.00
1742.50
1745.75
1749.00
1752.25
1755.50
1758.75
1762.00
1765.25
1768.50
1772.00
1775.25
1778.50
1781.75
1785.00
1788.25
1791.50
1794.75
1798.00
1801.25
1804.50
1807.75
1811.00
1814.25
1817.50
1820.75
1824.00
1827.25
1830.50
1833.75
1837.00
1840.25
1843.50
1846.75
1850.00
1853.25
1856.50
1859.75
1863.00
1866.25
1869.50
1872.75
1875.00
1878.25
1881.50
1884.75
1888.00
1891.25
1894.50
1897.75
1901.00
1904.25
1907.50
1910.75
1914.00
1917.25
1920.50
1923.75
1927.00
1930.25
1933.50
1936.75
1940.00
1943.25
1946.50
1949.75
1953.00
1956.25
1959.50
1962.75
1966.00
1969.25
1972.50
1975.75
1979.00
1982.25
1985.50
1988.75
1992.00
1995.25
1998.50
2001.75
2005.00
2008.25
2011.50
2014.75
2018.00
2021.25
2024.50
2027.75
2031.00
2034.25
2037.50
2040.75
2044.00
2047.25
2050.50
2053.75
2057.00
2060.25
2063.50
2066.75
2070.00
2073.25
2076.50
2079.75
2083.00
2086.25
2089.50
2092.75
2096.00
2099.25
2102.50
2105.75
2109.00
2112.25
2115.50
2118.75
2122.00
2125.25
2128.50
2131.75
2135.00
2138.25
2141.50
2144.75
2148.00
2151.25
2154.50
2157.75
2161.00
2164.25
2167.50
2170.75
2174.00
2177.25
2180.50
2183.75
2187.00
2190.25
2193.50
2196.75
2200.00

Writing stocks-google.txt


In [12]:
!nvcc -arch=sm_70 -std=c++14 calcula_variancia.cu -o calcula_variancia

In [13]:
!./calcula_variancia < stocks-google.txt

A variância das diferenças diárias é: 3168.44


In [2]:
%%writefile Calcula_dias_aumento.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/count.h>
#include <iostream>
#include <fstream>
#include <cmath>

// Functor para calcular a diferença entre elementos consecutivos
struct diff_functor {
    __host__ __device__
    float operator()(const float& a, const float& b) const {
        return b - a;
    }
};

// Functor para calcular o quadrado da diferença entre um valor e a média
struct square_diff_functor {
    const float mean;

    square_diff_functor(float _mean) : mean(_mean) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

// Função para verificar se o valor é positivo (aumento)
struct is_positive {
    __host__ __device__
    bool operator()(const float& x) const {
        return x > 0;
    }
};

// Função para ler o arquivo e preencher o vetor
thrust::host_vector<float> read_stock_data(const std::string& filename) {
    std::ifstream file(filename);
    thrust::host_vector<float> stocks;
    float value;

    if (file.is_open()) {
        while (file >> value) {
            stocks.push_back(value);
        }
        file.close();
    } else {
        std::cerr << "Erro ao abrir o arquivo." << std::endl;
    }

    return stocks;
}

int main() {
    // Ler o arquivo de texto com os preços de ações
    std::string filename = "stocks-google.txt";
    thrust::host_vector<float> h_stocks = read_stock_data(filename);

    if (h_stocks.size() < 2) {
        std::cerr << "Arquivo com poucos dados para calcular diferenças diárias." << std::endl;
        return 1;
    }

    // Copiar o vetor para o device
    thrust::device_vector<float> d_stocks = h_stocks;

    // Vetor de saída com um elemento a menos
    thrust::device_vector<float> d_ganho_diario(d_stocks.size() - 1);

    // Calcular a diferença diária
    thrust::transform(d_stocks.begin(), d_stocks.end() - 1,  // Intervalo do primeiro vetor (stocks[i])
                      d_stocks.begin() + 1,                  // Intervalo do segundo vetor (stocks[i+1])
                      d_ganho_diario.begin(),                // Vetor de saída (ganho_diario[i])
                      diff_functor());                       // Functor de diferença

    // Calcular a soma total do vetor de ganho_diario
    float soma = thrust::reduce(d_ganho_diario.begin(), d_ganho_diario.end(), 0.0f, thrust::plus<float>());

    // Calcular a média
    float media = soma / d_ganho_diario.size();

    // Usar thrust::transform_reduce para calcular a variância
    float variancia = thrust::transform_reduce(
        d_ganho_diario.begin(), d_ganho_diario.end(),   // Intervalo do vetor de diferenças
        square_diff_functor(media),                     // Functor para calcular o quadrado da diferença
        0.0f,                                           // Valor inicial da redução
        thrust::plus<float>()                           // Operador de soma
    );

    // Dividir pela quantidade de elementos para obter a variância
    variancia /= d_ganho_diario.size();

    // Contar quantos dias o preço aumentou (ganho_diario > 0)
    int num_dias_com_aumento = thrust::count_if(d_ganho_diario.begin(), d_ganho_diario.end(), is_positive());

    // Exibir o resultado
    std::cout << "A variância das diferenças diárias é: " << variancia << std::endl;
    std::cout << "Número de dias com aumento: " << num_dias_com_aumento << std::endl;

    return 0;
}


Overwriting Calcula_dias_aumento.cu


In [3]:
!nvcc -arch=sm_70 -std=c++14 Calcula_dias_aumento.cu -o Calcula_dias_aumento

In [7]:
!./Calcula_dias_aumento < stocks-google.txt

A variância das diferenças diárias é: 3168.44
Número de dias com aumento: 3056


In [8]:
%%writefile calcula_aumento_medio.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/count.h>
#include <thrust/replace.h>
#include <iostream>
#include <fstream>
#include <cmath>

// Functor para calcular a diferença entre elementos consecutivos
struct diff_functor {
    __host__ __device__
    float operator()(const float& a, const float& b) const {
        return b - a;
    }
};

// Functor para calcular o quadrado da diferença entre um valor e a média
struct square_diff_functor {
    const float mean;

    square_diff_functor(float _mean) : mean(_mean) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

// Função para verificar se o valor é negativo
struct is_negative {
    __host__ __device__
    bool operator()(const float& x) const {
        return x < 0;
    }
};

// Função para verificar se o valor é positivo (aumento)
struct is_positive {
    __host__ __device__
    bool operator()(const float& x) const {
        return x > 0;
    }
};

// Função para ler o arquivo e preencher o vetor
thrust::host_vector<float> read_stock_data(const std::string& filename) {
    std::ifstream file(filename);
    thrust::host_vector<float> stocks;
    float value;

    if (file.is_open()) {
        while (file >> value) {
            stocks.push_back(value);
        }
        file.close();
    } else {
        std::cerr << "Erro ao abrir o arquivo." << std::endl;
    }

    return stocks;
}

int main() {
    // Ler o arquivo de texto com os preços de ações
    std::string filename = "stocks-google.txt";
    thrust::host_vector<float> h_stocks = read_stock_data(filename);

    if (h_stocks.size() < 2) {
        std::cerr << "Arquivo com poucos dados para calcular diferenças diárias." << std::endl;
        return 1;
    }

    // Copiar o vetor para o device
    thrust::device_vector<float> d_stocks = h_stocks;

    // Vetor de saída com um elemento a menos
    thrust::device_vector<float> d_ganho_diario(d_stocks.size() - 1);

    // Calcular a diferença diária
    thrust::transform(d_stocks.begin(), d_stocks.end() - 1,  // Intervalo do primeiro vetor (stocks[i])
                      d_stocks.begin() + 1,                  // Intervalo do segundo vetor (stocks[i+1])
                      d_ganho_diario.begin(),                // Vetor de saída (ganho_diario[i])
                      diff_functor());                       // Functor de diferença

    // Calcular a soma total do vetor de ganho_diario
    float soma = thrust::reduce(d_ganho_diario.begin(), d_ganho_diario.end(), 0.0f, thrust::plus<float>());

    // Calcular a média
    float media = soma / d_ganho_diario.size();

    // Usar thrust::transform_reduce para calcular a variância
    float variancia = thrust::transform_reduce(
        d_ganho_diario.begin(), d_ganho_diario.end(),   // Intervalo do vetor de diferenças
        square_diff_functor(media),                     // Functor para calcular o quadrado da diferença
        0.0f,                                           // Valor inicial da redução
        thrust::plus<float>()                           // Operador de soma
    );

    // Dividir pela quantidade de elementos para obter a variância
    variancia /= d_ganho_diario.size();

    // Contar quantos dias o preço aumentou (ganho_diario > 0)
    int num_dias_com_aumento = thrust::count_if(d_ganho_diario.begin(), d_ganho_diario.end(), is_positive());

    // Substituir valores negativos de ganho_diario por zero
    thrust::replace_if(d_ganho_diario.begin(), d_ganho_diario.end(), is_negative(), 0.0f);

    // Calcular a soma dos valores positivos (após substituição)
    float soma_aumentos = thrust::reduce(d_ganho_diario.begin(), d_ganho_diario.end(), 0.0f, thrust::plus<float>());

    // Calcular o aumento médio apenas nos dias em que o preço subiu
    float aumento_medio = (num_dias_com_aumento > 0) ? soma_aumentos / num_dias_com_aumento : 0.0f;

    // Exibir o resultado
    std::cout << "A variância das diferenças diárias é: " << variancia << std::endl;
    std::cout << "Número de dias com aumento: " << num_dias_com_aumento << std::endl;
    std::cout << "Aumento médio nos dias em que o preço subiu: " << aumento_medio << std::endl;

    return 0;
}


Writing calcula_aumento_medio.cu


In [9]:
!nvcc -arch=sm_70 -std=c++14 calcula_aumento_medio.cu -o calcula_aumento_medio

In [12]:
!./calcula_aumento_medio < stocks-google.txt

A variância das diferenças diárias é: 3168.44
Número de dias com aumento: 3056
Aumento médio nos dias em que o preço subiu: 3.15251
